# Evaluation of GLTR for Polar Bear Paper

In [ ]:
%matplotlib widget
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import random as rnd
from fau_colors import cmaps
from matplotlib import patches as mpatches
from pb_evaluation import PBEvaluation

colors = cmaps.faculties_all

## Process GLTR Feature Vectors

Use the PBEvaluation class to get Rank-1 and mAP metrics for the GLTR feature vectors  
Save the resulting files into csv files

In [ ]:
# def paths
dataset_path = '...'
feature_path = '...'
epoch = 250

In [ ]:
# Setup
fold_file_r1, fold_file_map = None, None
r1_data, map_data = [], []
mAPs, rank1s = [], []
zoos = None

In [ ]:
# go through all the folds
for i in range(5):
    # read fold
    print(f'\nEvaluate Fold {i + 1}')
    fold_dir = os.path.join(feature_path, f'fold_{i + 1}')
    fold_info_file = f'track_fold_info_{i + 1}.csv'

    # create eval class
    pb_eval = PBEvaluation(dataset_path, fold_dir, fold_info_file)

    # get zoos
    if zoos is None:
        zoos = pb_eval.zoos

    mAP, CMC, mAP_per_id, CMC_per_id = pb_eval.eval_single_feat_file(epoch, f'features_pb_fold_{i + 1}')
    mAPs.append(mAP)
    rank1s.append(CMC[0])

    # add to data arrays
    map_data.append([i, mAP] + [val for _, val in sorted(mAP_per_id.items())] + [val for _, val in pb_eval.map_per_zoo.items()])
    r1_data.append([i, CMC[0]] + [val[0] for _, val in sorted(CMC_per_id.items())] + [val for _, val in pb_eval.r1_per_zoo.items()])

print('\nMean values of all 5 folds:')
fold_mAP = np.sum(mAPs) / len(mAPs)
fold_r1 = np.sum(rank1s) / len(rank1s)
print(f'mAP: {fold_mAP}')
print(f'R1: {fold_r1}')

# save data
animal_ids = np.ravel([z[1:] for z in zoos])
col = ['fold', 'mean'] + [str(a_id) for a_id in animal_ids] + [z[0] for z in zoos]
fold_file_r1 = pd.DataFrame(r1_data, columns=col)
fold_file_map = pd.DataFrame(map_data, columns=col)

fold_file_r1.to_csv('r1_5_fold_pb_paper.csv', index=False, sep=';')
fold_file_map.to_csv('map_5_fold_pb_paper.csv', index=False, sep=';')
print('--- saved ---')

## Plot GLTR-based Graphs

Prelim: GLTR mean and spread  
Mean of every fold  
Zoos comparison  
ID comparison  
TODO: Training curve (epoch)  
TODO: Confusion Matrix  

In [ ]:
# load data if nessesary
eval_files = ''

fold_file_r1 = pd.read_csv(os.path.join(eval_files, 'r1_5_fold_pb_paper.csv'), sep=';')
fold_file_map = pd.read_csv(os.path.join(eval_files, 'map_5_fold_pb_paper.csv'), sep=';')

In [ ]:
def def_plot(metric, x_title, x_ticks):
    # plot graph
    _, ax = plt.subplots()
    if metric == 'r1':
        plt.ylabel('Rank-1 (%)')
    else:
        plt.ylabel('mAP (%)')
    plt.xlabel(x_title)
    ax.set_axisbelow(True)
    plt.grid(axis='y')

    # set x values
    ax.set_xticks(range(len(x_ticks)))
    ax.set_xticklabels(x_ticks)
    ax.set_ylim([0.0, 1.0])
    ax.set_yticks([i / 100 for i in range(0, 101, 10)])
    ax.set_yticklabels(range(0, 101, 10))

    return ax

In [ ]:
def plot_comp(metric, data):
    # TODO
    mean_fold = data['mean'].mean()
    std_fold = data['mean'].std(ddof=0)

    ax = def_plot(metric, 'Approaches', ['GLTR'])

    plt.bar([0], mean_fold, align='center', color=colors[2], yerr=std_fold, ecolor='black', capsize=10)
    plt.show()

In [ ]:
def plot_fold(metric, data):
    fold_mean = list(data['mean'])

    ax = def_plot(metric, 'Folds', [str(i) for i in range(1, 6)])
    colors_n = colors[:4] + [colors[6]]

    plt.bar(list(range(5)), fold_mean, align='center', color=colors_n)
    plt.show()

In [ ]:
def plot_zoos(metric, data):
    zoos = list(data.columns)[-7:]

    mean_fold = data[zoos].mean()
    std_fold = data[zoos].std(ddof=0)

    ax = def_plot(metric, 'Zoos', zoos)

    plt.bar(list(range(zoos)), mean_fold, align='center', color=colors, yerr=std_fold, ecolor='black', capsize=10)
    plt.show()

In [ ]:
def plot_ids(metric, data):
    ids = list(data.columns)[2:-7]

    mean_fold = data[ids].mean()
    std_fold = data[ids].std(ddof=0)

    ax = def_plot(metric, 'Individuals', ids)

    plt.bar(list(range(ids)), mean_fold, align='center', color=colors, yerr=std_fold, ecolor='black', capsize=10)
    plt.show()